In [1]:
import bw2data as bd
import bw2io as bi
from pprint import pprint

bd.projects.set_current("optimex")
bd.databases

Databases dictionary with 8 object(s):
	biosphere3
	ecoinvent-3.9.1-biosphere
	ecoinvent-3.9.1-cutoff
	ei3.9.1-SSP2-RCP19-2020
	ei3.9.1-SSP2-RCP19-2030
	ei3.9.1-SSP2-RCP19-2040
	ei3.9.1-SSP2-RCP19-2050
	foreground

In [2]:
# prospective databases
db_2020_name = "ei3.9.1-SSP2-RCP19-2020"
db_2030_name = "ei3.9.1-SSP2-RCP19-2030"
db_2040_name = "ei3.9.1-SSP2-RCP19-2040"

eidb = bd.Database("ecoinvent-3.9.1-cutoff")
db_2020 = bd.Database(db_2020_name)
db_2030 = bd.Database(db_2030_name)
db_2040 = bd.Database(db_2040_name)

In [3]:
# dict_electricity = {
#     "wind": bd.get_activity((eidb, "24ec283aa11d98c3832b5556ac811cd2")),
#     "lignite": bd.get_activity((eidb, "f34dea30e935bc65b1d20422c30b0592")),
#     "gas": bd.get_activity((eidb, "383cb0b4ff9abe80582141530301591a")),
#     "solar": bd.get_activity((eidb, "0105e3a970d2bf162c500e0285a4b3a9"))
# }

dict_el_org_activities = {
    "wind": [act for act in db_2020 if "electricity production, wind, >3MW turbine, onshore" in act["name"] and "DE" in act["location"]][0],
    "lignite": [act for act in db_2020 if "electricity production, lignite" in act["name"] and "DE" in act["location"]][0],
    "gas": [act for act in db_2020 if "electricity production, natural gas, combined cycle power plant" in act["name"] and "DE" in act["location"]][0],
    "solar": [act for act in db_2020 if "electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted" in act["name"] and "DE" in act["location"]][0]
}

In [4]:
for name, act in dict_el_org_activities.items():
    print(name, act)
    for i, exchange in enumerate(act.exchanges()):
        print(f'{i} (flowtype {exchange["type"]}): {exchange}')
    print("\n")
    

wind 'electricity production, wind, >3MW turbine, onshore' (kilowatt hour, DE, None)
0 (flowtype production): Exchange: 1.0 kilowatt hour 'electricity production, wind, >3MW turbine, onshore' (kilowatt hour, DE, None) to 'electricity production, wind, >3MW turbine, onshore' (kilowatt hour, DE, None)>
1 (flowtype production): Exchange: -4.9149577534990385e-05 kilogram 'market for waste mineral oil' (kilogram, Europe without Switzerland, None) to 'electricity production, wind, >3MW turbine, onshore' (kilowatt hour, DE, None)>
2 (flowtype technosphere): Exchange: 6.9346848263762695e-09 unit 'market for wind turbine, 4.5MW, onshore' (unit, GLO, None) to 'electricity production, wind, >3MW turbine, onshore' (kilowatt hour, DE, None)>
3 (flowtype technosphere): Exchange: 6.9346848263762695e-09 unit 'market for wind turbine network connection, 4.5MW, onshore' (unit, GLO, None) to 'electricity production, wind, >3MW turbine, onshore' (kilowatt hour, DE, None)>
4 (flowtype technosphere): Exchan

In [5]:
if "foreground" in bd.databases:
    del bd.databases["foreground"] # to make sure we create the foreground from scratch
foreground = bd.Database("foreground")
foreground.write({}) 
fg = bd.Database("foreground")

In [6]:
def create_foreground_activity(activity: bd.backends.proxies.Activity, foreground: bd.backends.base.SQLiteBackend):
    # if foreground.get(activity["code"]):
    #     return foreground.get(activity["code"])

    foreground.new_node(
        code=activity["code"],
        name=activity["name"],
        unit=activity["unit"],
        location=activity["location"],
        reference_product=activity["reference product"],
    ).save()

    new_activity = foreground.get(activity["code"])
    
    for exchange in activity.exchanges():
        if exchange["type"] == "production":
            # to avoid multifunctionality we only create a prodction exchange for the reference product which has an amount of 1
            if exchange["amount"] != 1:
                continue
            new_activity.new_edge(
                input=new_activity,
                amount=exchange["amount"],
                type="production"
            ).save()
        else:
            new_activity.new_edge(
                input=exchange.input,
                amount=exchange["amount"],
                type=exchange["type"]
            ).save()
    
    return new_activity

for name, act in dict_el_org_activities.items():
    create_foreground_activity(act, fg)

c:\Users\LaytonSurface\OneDrive\Desktop\LTT_repos\time-explicit-life-cycle-optimization\venv\Lib\site-packages\bw2data\backends\typos.py:91: UserWarning: Possible incorrect activity key found: Given `reference_product` but `reference product` is more common
  warnings.warn(


In [7]:
[act for act in fg]

['electricity production, natural gas, combined cycle power plant' (kilowatt hour, DE, None),
 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted' (kilowatt hour, DE, None),
 'electricity production, lignite' (kilowatt hour, DE, None),
 'electricity production, wind, >3MW turbine, onshore' (kilowatt hour, DE, None)]

In [8]:
from bw_temporalis import TemporalDistribution, easy_timedelta_distribution
import numpy as np

LIFETIME = 20

lifecycle_td = {}

lifecycle_td["construction"] = easy_timedelta_distribution(
    start=-4,
    end=0,
    resolution="Y",     # "Y": "Years", "M": "Months", "D": "Days", "h": "Hours" etc.
    steps=5,
    kind="uniform", # available kinds: "triangular", "uniform", "normal"
    param=-1,
)

lifecycle_td["operation"] = easy_timedelta_distribution(
    start=0,  # (inclusive)
    end=LIFETIME,  # (inclusive)
    resolution="Y",
    steps=(LIFETIME + 1),  # Includes both start and end
    kind="uniform",  
)

lifecycle_td["end_of_life"]= TemporalDistribution(
    date=np.array([LIFETIME + 1], dtype="timedelta64[Y]"), amount=np.array([1])  # if you build a TD manually, make sure that length of date array == length of amount array, and the sum of elements in the amount array == 1
)

In [9]:
construction_keywords = ["construct", "build", "install", "erect", "assemble", "fabricate", "manufacture"]
operation_keywords = ["use", "operation", "maintenance", "yearly"]
end_of_life_keywords = ["end of life", "decommission", "disposal", "recycling"]

def map_exchange_preliminary_to_lifecycle_phase(activity: bd.backends.proxies.Activity) -> dict[str, str]:
    mapping = {}
    for exc in activity.exchanges():
        if exc["type"] =="production":
            continue
        exc_name = exc.input["name"].lower()
        exc_comment = exc.input.get("comment", "").lower()
        lifecycle_phase = "unknown"
        
        # Determine the lifecycle phase based on the exchange name
        if any(keyword in exc_name or exc_comment for keyword in construction_keywords):
            lifecycle_phase = "construction"
        elif any(keyword in exc_name or exc_comment  for keyword in operation_keywords):
            lifecycle_phase = "operation"
        elif any(keyword in exc_name or exc_comment for keyword in end_of_life_keywords):
            lifecycle_phase = "end_of_life"

        if exc_name not in mapping:
            mapping[exc_name] = lifecycle_phase
    return mapping

In [10]:
dict_el_fg_activities = {}
for name, act in dict_el_org_activities.items():
    dict_el_fg_activities[name] = fg.get(act["code"])

activity_to_map = dict_el_fg_activities["wind"]

pprint(activity_to_map)
pprint('exchanges: ')
pprint(map_exchange_preliminary_to_lifecycle_phase(activity_to_map))

'electricity production, wind, >3MW turbine, onshore' (kilowatt hour, DE, None)
'exchanges: '
{'energy, kinetic (in wind), converted': 'unknown',
 'market for lubricating oil': 'construction',
 'market for transport, freight, lorry 7.5-16 metric ton, euro3': 'construction',
 'market for wind turbine network connection, 4.5mw, onshore': 'construction',
 'market for wind turbine, 4.5mw, onshore': 'construction'}


Prompt:
For the given mapping of an acitivity, assign each exchange one of the lifephases construction, operation or end_of_life based on the context of its name instead of unknown. Dont write any code, just do it manually by general knowledge. When in doubt assume the exchange happens in the operation phase. Write it down in a new dict of the form - key: exchange name, value: lifecycle phase

In [11]:
mapping = {}
mapping["gas"] = {
    'acenaphthene': 'operation',
    'acetaldehyde': 'operation',
    'acetic acid': 'operation',
    'arsenic ion': 'operation',
    'benzene': 'operation',
    'benzo(a)pyrene': 'operation',
    'beryllium ii': 'operation',
    'butane': 'operation',
    'cadmium ii': 'operation',
    'carbon dioxide, fossil': 'operation',
    'carbon monoxide, fossil': 'operation',
    'chromium iii': 'operation',
    'cobalt ii': 'operation',
    'dinitrogen monoxide': 'operation',
    'dioxins, measured as 2,3,7,8-tetrachlorodibenzo-p-dioxin': 'operation',
    'ethane': 'operation',
    'formaldehyde': 'operation',
    'hexane': 'operation',
    'lead ii': 'operation',
    'manganese ii': 'operation',
    'mercury ii': 'operation',
    'methane, fossil': 'operation',
    'nickel ii': 'operation',
    'nitrogen oxides': 'operation',
    'pah, polycyclic aromatic hydrocarbons': 'operation',
    'particulate matter, < 2.5 um': 'operation',
    'pentane': 'operation',
    'propane': 'operation',
    'propionic acid': 'operation',
    'selenium iv': 'operation',
    'sulfur dioxide': 'operation',
    'toluene': 'operation',
    'water': 'operation',
    'water, cooling, unspecified natural origin': 'operation',
    'gas power plant construction, combined cycle, 400mw electrical': 'construction',
    'market for natural gas, high pressure': 'operation',
    'market for residue from cooling tower': 'operation',
    'market for water, completely softened': 'construction',
    'market for water, decarbonised': 'construction',
    'mercury ii': 'operation',
    'methane, fossil': 'operation',
    'nickel ii': 'operation',
    'nitrogen oxides': 'operation',
    'pah, polycyclic aromatic hydrocarbons': 'operation',
    'particulate matter, < 2.5 um': 'operation',
    'pentane': 'operation',
    'propane': 'operation',
    'propionic acid': 'operation',
    'selenium iv': 'operation',
    'sulfur dioxide': 'operation',
    'toluene': 'operation',
    'water': 'operation',
    'water, cooling, unspecified natural origin': 'operation'   
}
mapping["lignite"] = {
    'antimony ion': 'operation',
    'arsenic ion': 'operation',
    'barium ii': 'operation',
    'benzene': 'operation',
    'benzo(a)pyrene': 'operation',
    'boron': 'operation',
    'bromine': 'operation',
    'butane': 'operation',
    'cadmium ii': 'operation',
    'carbon dioxide, fossil': 'operation',
    'carbon monoxide, fossil': 'operation',
    'chromium iii': 'operation',
    'chromium vi': 'operation',
    'cobalt ii': 'operation',
    'copper ion': 'operation',
    'dinitrogen monoxide': 'operation',
    'dioxins, measured as 2,3,7,8-tetrachlorodibenzo-p-dioxin': 'operation',
    'ethane': 'operation',
    'formaldehyde': 'operation',
    'hydrocarbons, aliphatic, alkanes, unspecified': 'operation',
    'hydrocarbons, aliphatic, unsaturated': 'operation',
    'hydrochloric acid': 'operation',
    'hydrogen fluoride': 'operation',
    'iodine': 'operation',
    'lead ii': 'operation',
    'lead-210': 'operation',
    'lignite power plant construction': 'construction',
    'manganese ii': 'operation',
    'market for chlorine, liquid': 'operation',
    'market for lignite': 'operation',
    'market for lignite ash': 'operation',
    'market for nox retained, by selective catalytic reduction': 'operation',
    'market for residue from cooling tower': 'operation',
    'market for sox retained, in lignite flue gas desulfurisation': 'operation',
    'market for water, completely softened': 'operation',
    'market for water, decarbonised': 'operation',
    'mercury ii': 'operation',
    'methane, fossil': 'operation',
    'molybdenum vi': 'operation',
    'nickel ii': 'operation',
    'nitrogen oxides': 'operation',
    'pah, polycyclic aromatic hydrocarbons': 'operation',
    'particulate matter, < 2.5 um': 'operation',
    'particulate matter, > 10 um': 'operation',
    'particulate matter, > 2.5 um and < 10um': 'operation',
    'pentane': 'operation',
    'polonium-210': 'operation',
    'potassium-40': 'operation',
    'propane': 'operation',
    'propene': 'operation',
    'radium-226': 'operation',
    'radium-228': 'operation',
    'radon-220': 'operation',
    'radon-222': 'operation',
    'selenium iv': 'operation',
    'strontium': 'operation',
    'sulfur dioxide': 'operation',
    'thorium-228': 'operation',
    'thorium-232': 'operation',
    'toluene': 'operation',
    'uranium-238': 'operation',
    'vanadium v': 'operation',
    'water': 'operation',
    'water, cooling, unspecified natural origin': 'operation',
    'xylene': 'operation',
    'zinc ii': 'operation'
}
mapping["solar"] = {
    'energy, solar, converted': 'operation',
    'market for photovoltaic slanted-roof installation, 3kwp, multi-si, panel, mounted, on roof': 'construction',
    'market for tap water': 'operation',
    'market for wastewater, average': 'operation',
    'water': 'operation'
}
mapping["wind"] = {
    'energy, kinetic (in wind), converted': 'operation',
    'market for lubricating oil': 'operation',
    'market for transport, freight, lorry 7.5-16 metric ton, euro3': 'operation',
    'market for waste mineral oil': 'operation',
    'market for wind turbine network connection, 4.5mw, onshore': 'construction',
    'market for wind turbine, 4.5mw, onshore': 'construction'
}

In [12]:
def set_temporal_distribution_of_lifecycle(activity: bd.backends.proxies.Activity, mapping: dict[str, str], temporal_distribution: dict[str, TemporalDistribution]) -> bd.backends.proxies.Activity:
    for exc in activity.exchanges():
        if exc["type"] == "production":
            continue
        exc_name = exc.input["name"].lower()
        
        if exc_name in mapping:
            lifecycle_phase = mapping[exc_name]
        else:
            print(f"Could not find lifecycle phase for exchange: {exc_name}")
            continue
        exc["lifecycle_phase"] = lifecycle_phase
        exc["temporal_distribution"] = temporal_distribution[lifecycle_phase]
        exc.save()

for name, act in dict_el_fg_activities.items():
    set_temporal_distribution_of_lifecycle(act, mapping[name], lifecycle_td)



In [13]:
for exc in dict_el_fg_activities["solar"].exchanges():
    print(exc.as_dict())

{'output': ('foreground', '0105e3a970d2bf162c500e0285a4b3a9'), 'input': ('foreground', '0105e3a970d2bf162c500e0285a4b3a9'), 'amount': 1.0, 'type': 'production'}
{'output': ('foreground', '0105e3a970d2bf162c500e0285a4b3a9'), 'input': ('ei3.9.1-SSP2-RCP19-2020', '09a9ae111d33545514b77a45cbada56d'), 'amount': 1.4446525e-05, 'type': 'technosphere', 'lifecycle_phase': 'construction', 'temporal_distribution': TemporalDistribution instance with 5 values and total: 1}
{'output': ('foreground', '0105e3a970d2bf162c500e0285a4b3a9'), 'input': ('ei3.9.1-SSP2-RCP19-2020', '4fe1f2dae4830593ee2d608cb2d5ff2c'), 'amount': 0.006385364, 'type': 'technosphere', 'lifecycle_phase': 'operation', 'temporal_distribution': TemporalDistribution instance with 21 values and total: 1}
{'output': ('foreground', '0105e3a970d2bf162c500e0285a4b3a9'), 'input': ('ecoinvent-3.9.1-biosphere', '075e433b-4be4-448e-9510-9a5029c1ce94'), 'amount': 9.578046e-07, 'type': 'biosphere', 'lifecycle_phase': 'operation', 'temporal_distr

In [14]:
method = ("EF v3.1", "climate change", "global warming potential (GWP100)")

In [15]:
from datetime import datetime
from bw_timex import TimexLCA

database_date_dict = {
    db_2020_name : datetime.strptime("2020", "%Y"),
    db_2030_name : datetime.strptime("2030", "%Y"),
    db_2040_name : datetime.strptime("2040", "%Y"),
    "foreground": "dynamic",  # flag databases that should be temporally distributed with "dynamic"
}


In [16]:
import dill as pickle
import time
from tqdm import tqdm

tlca_results = {}

start_time = time.time()

for name, act in tqdm(dict_el_fg_activities.items()):
    print("Conducting TLCA for activity: ", act)
    tlca = TimexLCA({act: 1 }, method, database_date_dict)
    tlca_results[act] = {}
    print("Building timeline...")
    df_timeline = tlca.build_timeline()
    print("Performing LCI...")
    tlca.lci()
    print("Performing static LCA...")
    tlca.static_lcia()
    tlca_results[act]["static"] = tlca.score #kg CO2-eq
    print("Performing dynamic LCIA...")
    tlca.dynamic_lcia(metric="radiative_forcing", fixed_time_horizon=True)
    tlca_results[act]["dynamic"] = tlca.dynamic_score #W/m2 (radiative forcing)

    print("Saving TLCA object...")
    try:
        with open(f'tlca_{name}.pickle', 'wb') as f:
            pickle.dump(tlca, f)
    except Exception as e:
        print(e)

end_time = time.time()
runtime = end_time - start_time
runtime


  0%|          | 0/4 [00:00<?, ?it/s]

Conducting TLCA for activity:  'electricity production, wind, >3MW turbine, onshore' (kilowatt hour, DE, None)
Building timeline...


c:\Users\LaytonSurface\OneDrive\Desktop\LTT_repos\time-explicit-life-cycle-optimization\venv\Lib\site-packages\bw_timex\timex_lca.py:182: UserWarning: No edge filter function provided. Skipping all edges within background databases.
  warnings.warn(


Starting graph traversal
Calculation count: 3
Performing LCI...


c:\Users\LaytonSurface\OneDrive\Desktop\LTT_repos\time-explicit-life-cycle-optimization\venv\Lib\site-packages\bw_timex\timeline_builder.py:482: Warning: Reference date 2041-01-01 00:00:00 is higher than all provided dates. Data will be taken from the closest lower year.
  warnings.warn(
c:\Users\LaytonSurface\OneDrive\Desktop\LTT_repos\time-explicit-life-cycle-optimization\venv\Lib\site-packages\bw_timex\timeline_builder.py:482: Warning: Reference date 2042-01-01 00:00:00 is higher than all provided dates. Data will be taken from the closest lower year.
  warnings.warn(
c:\Users\LaytonSurface\OneDrive\Desktop\LTT_repos\time-explicit-life-cycle-optimization\venv\Lib\site-packages\bw_timex\timeline_builder.py:482: Warning: Reference date 2043-01-01 00:00:00 is higher than all provided dates. Data will be taken from the closest lower year.
  warnings.warn(
c:\Users\LaytonSurface\OneDrive\Desktop\LTT_repos\time-explicit-life-cycle-optimization\venv\Lib\site-packages\bw_timex\timeline_buil

...

Performing static LCA...
Performing dynamic LCIA...


c:\Users\LaytonSurface\OneDrive\Desktop\LTT_repos\time-explicit-life-cycle-optimization\venv\Lib\site-packages\bw_timex\dynamic_characterization.py:71: UserWarning: No custom dynamic characterization functions provided. Using default dynamic characterization functions based on IPCC AR6 meant to work with biosphere3 flows. The flows that are characterized are based on the selection of the initially chosen impact category: ('EF v3.1', 'climate change', 'global warming potential (GWP100)'). You can look up the mapping in the bw_timex.dynamic_characterizer.characterization_function_dict.
  warnings.warn(


Saving TLCA object...
cannot pickle 'SuperLU' object


100%|██████████| 4/4 [1:11:19<00:00, 1069.76s/it]


4279.037497282028